<span style="float:right"><i>Peter Norvig<br>5 January 2016<br>revised 18 May 2018</i></span>

# Four 4s, Five 5s, and Countdown to 2016

On January 1, 2016 Alex Bellos [posed](http://www.theguardian.com/science/2016/jan/04/can-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) this New Year's puzzle:


>Fill in the blanks so that this equation makes arithmetical sense:

> 10 ␣ 9 ␣ 8 ␣ 7 ␣ 6 ␣ 5 ␣ 4 ␣ 3 ␣ 2 ␣ 1 = 2016

> You are allowed to use *only* the four basic arithmetical operations: +, -, &times;, ÷. But brackets (parentheses) can be used wherever needed. So, for example, the solution could begin

> `(10 + 9) * (8` ...

> or

> `10 + (9 * 8)` ...

Let's see if we can solve this puzzle, and some of the related ones from Alex's [first](http://www.theguardian.com/science/2016/jan/04/can-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) and [second](http://www.theguardian.com/science/2016/jan/04/did-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) post.  We'll start with a  simpler version of the puzzle.

# Four Operations, No Brackets

Suppose for the moment we are not allowed to use brackets.  Then there are nine blanks,  each of which can be filled by one of four operators, so there are 9<sup>4</sup> = 262,144 possibilities, few enough that we can enumerate them all, using `itertools.product` to get sequences of operators, and then `str.format` to plug them into blanks, and then `eval` to evaluate the string:

In [1]:
import itertools
from functools import lru_cache # Used later

ops = next(itertools.product(('+', '-', '*', '/'), repeat=9))
ops

('+', '+', '+', '+', '+', '+', '+', '+', '+')

In [2]:
'10{}9{}8{}7{}6{}5{}4{}3{}2{}1'.format(*ops)

'10+9+8+7+6+5+4+3+2+1'

In [3]:
eval(_)

55

We need to catch errors such as dividing by zero, so I'll define a wrapper function, `evaluate`, to do that, and I'll define `countdown_no_brackets` to put the pieces together:

In [4]:
def evaluate(exp):
    "eval exp, or return None if there is an arithmetic error."
    try:
        return eval(exp)
    except ArithmeticError:
        return None

def countdown_no_brackets(target, operators=('+', '-', '*', '/')):
    "All solutions to the countdown puzzle (with no brackets)."
    exps = ('10{}9{}8{}7{}6{}5{}4{}3{}2{}1'.format(*ops)
            for ops in itertools.product(operators, repeat=9))
    return [exp for exp in exps if evaluate(exp) == target]

In [5]:
countdown_no_brackets(2016)

[]

Too bad; we did all that work and didn't find a solution. What years *can* we find solutions for? I'll modify `countdown_no_brackets` to take a collection of target years rather than a single one, and return a dict of the form `{year: 'expression'}` for each expression that evaluates to one of the target years. I'll also generalize it to allow any format string, not just the countdown string.

In [6]:
def countdown_no_brackets(targets, operators=('+', '-', '*', '/'), 
                      fmt='10{}9{}8{}7{}6{}5{}4{}3{}2{}1'):
    "All solutions to the countdown puzzle (with no brackets)."
    exps = (fmt.format(*ops)
            for ops in itertools.product(operators, repeat=fmt.count('{}')))
    return {int(evaluate(exp)): exp for exp in exps if evaluate(exp) in targets}

countdown_no_brackets(range(1900, 2100))

{1979: '10*9*8+7*6*5*4*3/2-1',
 1980: '10*9*8+7*6*5*4*3/2/1',
 1981: '10*9*8+7*6*5*4*3/2+1',
 2013: '10*9*8*7/6/5*4*3-2-1',
 2014: '10*9*8*7/6/5*4*3-2/1',
 2015: '10*9*8*7/6/5*4*3-2+1',
 2017: '10*9*8*7/6/5*4*3+2-1',
 2018: '10*9*8*7/6/5*4*3+2/1',
 2019: '10*9*8*7/6/5*4*3+2+1'}

Interesting: in the 20th and 21st centuries, there are only two "golden eras" where the countdown equation works: the three year period centered on 1980, and the seven year period that is centered on 2016, but omits 2016. 

# Four Operations, With Brackets

What is the problem I want to solve? I want to know if I can create an expression whose value is 2016. But to get there I'll solve a more general problem: given a sequence of numbers, like `(10, 9, 8)`, what expressions can I make with them?
I'll define  `expressions(numbers)` to return a dict of `{value: expression}` 
for all expressions (strings) whose numeric value is `value`, and can be made from `numbers`, for example:

    expressions((10,)) ⇒ {10: '10'}
    expressions((9, 8)) ⇒ {1: '(9-8)', 1.125: '(9/8)', 17: '(9+8)', 72: '(9*8)'}

I'll use the idea of [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming): break the problem down into simpler subparts, compute an answer for each subpart, and remember intermediate results so we don't need to re-compute them later. How do we break the problem into parts? `expressions((10, 9, 8))` should consist of all the ways of splitting `(10, 9, 8)` into two parts, finding all the expressions that can be made with each part, and combining pairs of expressions with any of the four operators:

    expressions((10, 9, 8)) ⇒ {11: '(10+(9-8))', 27: '(10+(9+8))', 720: '(10*(9*8))', ...}

First the function `splits`:

In [7]:
c10 = (10, 9, 8, 7, 6, 5, 4, 3, 2, 1)

def splits(sequence):
    "Split sequence into two non-empty parts, in all ways."
    return [(sequence[:i], sequence[i:]) 
            for i in range(1, len(sequence))]

In [8]:
splits(c10)

[((10,), (9, 8, 7, 6, 5, 4, 3, 2, 1)),
 ((10, 9), (8, 7, 6, 5, 4, 3, 2, 1)),
 ((10, 9, 8), (7, 6, 5, 4, 3, 2, 1)),
 ((10, 9, 8, 7), (6, 5, 4, 3, 2, 1)),
 ((10, 9, 8, 7, 6), (5, 4, 3, 2, 1)),
 ((10, 9, 8, 7, 6, 5), (4, 3, 2, 1)),
 ((10, 9, 8, 7, 6, 5, 4), (3, 2, 1)),
 ((10, 9, 8, 7, 6, 5, 4, 3), (2, 1)),
 ((10, 9, 8, 7, 6, 5, 4, 3, 2), (1,))]

Now the function `expressions`:

In [9]:
@lru_cache()
def expressions(numbers: tuple) -> dict:
    "Return {value: expr} for all expressions that can be made from numbers."
    if len(numbers) == 1: 
        return {numbers[0]: str(numbers[0])}
    else: 
        result = {}
        for (Lnums, Rnums) in splits(numbers):
            for (L, R) in pairs(expressions(Lnums), expressions(Rnums)):
                Lexp =  '(' + expressions(Lnums)[L]
                Rexp =        expressions(Rnums)[R] + ')'
                if R != 0: 
                    result[L / R] = Lexp + '/' + Rexp
                result[L * R] = Lexp + '*' + Rexp
                result[L - R] = Lexp + '-' + Rexp
                result[L + R] = Lexp + '+' + Rexp
        return result
    
def pairs(left, right): return ((L, R) for L in left for R in right)

For example, at one point in a call to `expressions((10, 9, 8))` we would have:

     Lnums, Rnums = (10),   (9, 8)
     L,     R     = 10,     1
     Lexp,  Rexp  = '(10',  '(9-8))'

This would lead to us adding the following four entries to `result`:

     result[10] = '(10*(9-8))'
     result[9]  = '(10-(9-8))'
     result[11] = '(10+(9-8))'
     result[10] = '(10/(9-8))'
     
The decorator `@lru_cache` takes care of storing the intermediate results. Rather than catching errors, we just avoid division by 0.

Let's give it a try:

In [10]:
expressions((10,))

{10: '10'}

In [11]:
expressions((9, 8))

{1: '(9-8)', 1.125: '(9/8)', 17: '(9+8)', 72: '(9*8)'}

In [12]:
expressions((10, 9, 8))

{-62: '(10-(9*8))',
 -7: '((10-9)-8)',
 -6.888888888888889: '((10/9)-8)',
 0.125: '((10-9)/8)',
 0.1388888888888889: '((10/9)/8)',
 0.5882352941176471: '(10/(9+8))',
 2.375: '((10+9)/8)',
 8: '((10-9)*8)',
 8.875: '(10-(9/8))',
 8.88888888888889: '((10/9)*8)',
 9: '((10-9)+8)',
 9.11111111111111: '((10/9)+8)',
 10.0: '(10*(9-8))',
 11: '((10+9)-8)',
 11.125: '(10+(9/8))',
 11.25: '((10*9)/8)',
 27: '((10+9)+8)',
 82: '((10*9)-8)',
 98: '((10*9)+8)',
 152: '((10+9)*8)',
 170: '(10*(9+8))',
 720: '((10*9)*8)'}

That looks reasonable. Let's solve the whole puzzle.

# Countdown to 2016: A Solution

In [13]:
%time expressions(c10)[2016]

CPU times: user 28.3 s, sys: 728 ms, total: 29 s
Wall time: 29.2 s


'(((((((10+((9*8)*7))-6)-5)*4)+3)+2)-1)'

We have an answer! And in seconds, not hours, thanks to dynamic programming! Here are solutions for nearby years:

In [14]:
{y: expressions(c10)[y] for y in range(2010, 2025)}

{2010: '((((10*((9+((8+7)*6))+(5/4)))+3)*2)-1)',
 2011: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)-1)',
 2012: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)*1)',
 2013: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)+1)',
 2014: '((((10-(9*8))*(7-(6*((5+4)+3))))/2)-1)',
 2015: '((((((((10*9)+8)-7)*(6+5))+4)+3)*2)-1)',
 2016: '(((((((10+((9*8)*7))-6)-5)*4)+3)+2)-1)',
 2017: '(((((((10+((9*8)*7))-6)-5)*4)+3)+2)*1)',
 2018: '(((((((10+((9*8)*7))-6)-5)*4)+3)+2)+1)',
 2019: '(((((((((10*9)*8)*7)/6)/5)*4)*3)+2)+1)',
 2020: '((((10+((((9*8)*7)*(6-5))*4))-3)-2)-1)',
 2021: '((((10+((((9*8)*7)*(6-5))*4))-3)-2)*1)',
 2022: '((((10+((((9*8)*7)*(6-5))*4))-3)-2)+1)',
 2023: '(((((10*9)+8)*((7*6)-(5/(4+3))))/2)*1)',
 2024: '(((((10*9)+8)*((7*6)-(5/(4+3))))/2)+1)'}

# Four 4s

Alex Bellos continues with a related puzzle:
    
> The most famous “fill in the gaps in the equation” puzzle is known as the [four fours](https://en.wikipedia.org/wiki/Four_fours), because every equation is of the form

> 4 ␣ 4 ␣ 4 ␣ 4 = X.

>In the classic form of the puzzle you must find a solution for X = 0 to 9 using just addition, subtraction, multiplication and division.

This puzzle is "most famous" because it goes back to a [1914 publication by W. Ball](https://archive.org/details/mathematicalrecr00ball). The "0 to 9" solution is easy:

In [15]:
{i: expressions((4, 4, 4, 4))[i] for i in range(10)}

{0: '(((4-4)-4)+4)',
 1: '(((4/4)-4)+4)',
 2: '((4/(4+4))*4)',
 3: '(((4+4)+4)/4)',
 4: '(((4-4)*4)+4)',
 5: '(((4*4)+4)/4)',
 6: '(((4+4)/4)+4)',
 7: '((4-(4/4))+4)',
 8: '(((4+4)+4)-4)',
 9: '(((4/4)+4)+4)'}

# New Mathematical Operations

Bellos then writes:
    
> If you want to show off, you can introduce **new mathematical operations** such as powers, square roots, concatenation and decimals, ... or use the factorial symbol, `!`.

Bellos is suggesting the following operations:

- **Powers**: `(4 ^ 4)` is 4 to the fourth power, or 256.
- **Square roots:** `√4` is the square root of 4, or 2.
- **Concatenation:** `44` is forty-four.
- **Decimals:** `4.4` is four and four tenths.
- **Factorials** `4!` is 4 × 3 × 2 × 1, or 24.

There are some complications to deal with:

- **Irrationals**: `√2` is an irrational number; so we can't do exact rational arithmetic.
- **Imaginaries**: `√-1` is an imaginary number, but Python gives a `ValueError` for `sqrt(-1)`.
- **Overflow**: `(10. ^ (9. ^ 8.))`, as a `float`, gives an `OverflowError`.
- **Out of memory**: [`(10 ^ (9 ^ (8 * 7)))`](http://www.wolframalpha.com/input/?i=10+%5E+9+%5E+56), as an `int`, gives an `OutOfMemoryError` (even if your memory uses every atom on Earth).
- **Infinite expressions**: We could add an infinite number of square root and/or factorial signs to any expression: `√√√√√√(4!!!!!!!!)`...
- **Round-off error**: `(49*(1/49))` evaluates to `0.9999999999999999`, not `1`.

We could try to manage this with *symbolic algebra*, perhaps using [SymPy](http://www.sympy.org/en/index.html), but that seems complicated, so instead I will:
- Arbitrarily limit expressions to two nested unary operations to avoid infinite expressions.
- Use floats for all computation (to avoid out of memory errors), and accept that there are some approximations. 

I'll define the function `do` to do an arithmetic computation, catch any errors, and try to correct round-off errors. The idea is that since my expressions start with integers, any result that is very close to an integer is probably actually that integer. So I'll correct `(49*(1/49))` to be `1.0`. Of course, an expression like `(1+(10^-99))` is also very close to `1.0`, but it should not be rounded off. Instead, I'll try to avoid such expressions by silently dropping any value that is outside the range of 10<sup>-10</sup> to 10<sup>10</sup> (except of course I will accept an exact 0).

In [16]:
from operator import add, sub, mul, truediv
from math import sqrt, factorial
from collections import defaultdict, Counter

def do(op, *args): 
    "Return op(*args), trying to correct for roundoff error, or `None` on Error or too big/small."
    try:
        val = op(*args)
        return (val        if val == 0 else
                None       if not (1e-10 < abs(val) < 1e10) else
                round(val) if abs(val - round(val)) < 1e-12 else 
                val)
    except (ArithmeticError, ValueError):
        return None

In [17]:
assert do(truediv, 12, 4) == 3 and do(truediv, 12, 0) == None
assert do(mul, 49, do(truediv, 1, 49)) == 1
assert do(pow, 10, -99) == None

I'll take this opportunity to refactor `expressions` to have these subfunctions:
- `digit_expressions(result, numbers)`: returns a dict like {0.44: '.44', 4.4: '4.4', 44.0: '44'}
- `add_unary_expressions(result)`: adds expressions like `√44` and `4!` and return `result`. 
- `add_binary_expressions(result, numbers)`: adds expressions like `4+√44` and `4^4`  and return `result`.

In [18]:
@lru_cache(1000)
def expressions(numbers):
    "Return {value: expr} for all expressions that can be made from numbers."
    return add_unary_expressions(add_binary_expressions(digit_expressions(numbers), numbers))

def digit_expressions(numbers) -> dict:
    "All ways of making numbers from these numbers, in order, and a decimal point."
    D = ''.join(map(str, numbers))
    exps = [(D[:i] + '.' + D[i:]).rstrip('.')
            for i in range(len(D) + 1)
            if not (D.startswith('0') and i > 1)]
    return {float(exp): exp for exp in exps}
      
def add_binary_expressions(result, numbers) -> dict:
    "Add binary expressions by splitting numbers and combining with an op."
    for (Lnums, Rnums) in splits(numbers):
        for (L, R) in pairs(expressions(Lnums), expressions(Rnums)):
            Lexp = '(' + expressions(Lnums)[L]
            Rexp =       expressions(Rnums)[R] + ')'
            assign(result, do(truediv, L, R), Lexp + '/' + Rexp)
            assign(result, do(mul,     L, R), Lexp + '*'  + Rexp)
            assign(result, do(add,     L, R), Lexp + '+'  + Rexp)
            assign(result, do(sub,     L, R), Lexp + '-'  + Rexp)
            if -10 <= R <= 10 and (L > 0 or int(R) == R):
                assign(result, do(pow, L, R), Lexp + '^' + Rexp)
    return result
                
def add_unary_expressions(result, nesting_level=2) -> dict:
    "Add unary expressions: -v, √v and v!"
    for _ in range(nesting_level):
        for v in tuple(result):
            exp = result[v]
            if -v not in result:
                assign(result, -v, '-' + exp)
            if 0 < v <= 100 and 120 * v == round(120 * v): 
                assign(result, sqrt(v), '√' + exp)
            if 3 <= v <= 6 and v == int(v):
                assign(result, factorial(v), exp + '!')
    return result

The  function `assign` will silently drop expressions whose value is `None`, and if two expressions have the same value, `assign` keeps the one with the lower "weight"&mdash;a measure of the characters in the expression. This shows simpler expressions in favor of complex ones: for four 4s, the entry for `0` will be `44-44`, not something like `-√((-√4*--4)*(-√4--√4))`.

In [19]:
def assign(result, value, exp): 
    "Assign result[value] = exp, unless we already have a lighter exp, or value is too extreme."
    if (value is not None 
        and (value not in result or weight(exp) < weight(result[value]))): 
        result[value] = exp
        
PENALTIES = defaultdict(int, {'√':7, '!':5, '.':1, '^':3, '/':1, '-':1})
        
def weight(exp) -> int: return len(exp) + sum(PENALTIES[c] for c in exp)

I'll define a function to create a table of consecutive integers (starting at 0) that can be made by a sequence of numbers:

In [20]:
def show(numbers, clear=True):
    "Print a table of expressions for all consecutive integers i from 0 up to the first unmakeable integer."
    if clear: expressions.cache_clear() # To free up memory
    print('{:,} entries for {}\n'.format(len(expressions(numbers)), numbers))
    for i in itertools.count(0): # All integers from 0
        if i not in expressions(numbers):
            return i
        print('{:4} = {}'.format(i, unbracket(expressions(numbers)[i])))
              
def unbracket(exp):
    "Strip outer parens from exp, if they are there"
    return (exp[1:-1] if exp.startswith('(') and exp.endswith(')') else exp)

In [21]:
%time show((4, 4, 4, 4))

705,959 entries for (4, 4, 4, 4)

   0 = 44-44
   1 = 44/44
   2 = 4*(4/(4+4))
   3 = (4+(4+4))/4
   4 = 4+(4*(4-4))
   5 = (4+(4*4))/4
   6 = 4+((4+4)/4)
   7 = (44/4)-4
   8 = 4+(4+(4-4))
   9 = 4+(4+(4/4))
  10 = 44/4.4
  11 = (4/.4)+(4/4)
  12 = (4+44)/4
  13 = 4!-(44/4)
  14 = (4+(.4*4))/.4
  15 = 4+(44/4)
  16 = .4*(44-4)
  17 = (4*4)+(4/4)
  18 = .4+(.4*44)
  19 = (4+(4-.4))/.4
  20 = 4*(4+(4/4))
  21 = (4+4.4)/.4
  22 = √4*(44/4)
  23 = ((4!*4)-4)/4
  24 = 4+(4+(4*4))
  25 = (4+(4!*4))/4
  26 = (4/.4)+(4*4)
  27 = 4!+(4-(4/4))
  28 = 44-(4*4)
  29 = 4+(4/(.4*.4))
  30 = (4+(4+4))/.4
  31 = 4!+((4!+4)/4)
  32 = (4*4)+(4*4)
  33 = 4!+((4-.4)/.4)
  34 = 44-(4/.4)
  35 = 4!+(44/4)
  36 = 44-(4+4)
  37 = ((.4+4!)/.4)-4!
  38 = 44-(4!/4)
  39 = ((4*4)-.4)/.4
  40 = 44-√(4*4)
  41 = (.4+(4*4))/.4
  42 = √4+(44-4)
  43 = 44-(4/4)
  44 = 4+(44-4)
  45 = (4/4)+44
  46 = 4+(44-√4)
  47 = 4!+(4!-(4/4))
  48 = 4*(4+(4+4))
  49 = (√4/.4)+44
  50 = (4+(4*4))/.4
  51 = (.4+(4!-4))/.4
  52 = 4+

73

We can also solve the "2016 with four fours" puzzle:

In [22]:
expressions((4, 4, 4, 4))[2016]

'(4!*(4!+(4!/.4)))'

In a [separate video](https://www.youtube.com/embed/Noo4lN-vSvw), Alex Bellos shows  how to form **every** integer from 0 to infinity using four 4s, if the square root and `log` functions are allowed. The solution comes from Paul Dirac (although [Dirac originally developed it](https://nebusresearch.wordpress.com/2014/04/18/how-dirac-made-every-number/) for the "four 2s" problem).

Donald Knuth has [conjectured](https://www.tandfonline.com/doi/abs/10.1080/0025570X.1964.11975546) that with floor, square root and factorial, you can make any positive integer with just **one** 4.

Below are some popular variants:

# Four 2s

In [23]:
%time show((2, 2, 2, 2))

109,747 entries for (2, 2, 2, 2)

   0 = 22-22
   1 = 22/22
   2 = 2+(2*(2-2))
   3 = (2+(2*2))/2
   4 = .2*(22-2)
   5 = 2+(2+(2/2))
   6 = 2*(2+(2/2))
   7 = 2+(2/(.2*2))
   8 = 2+(2+(2*2))
   9 = (22/2)-2
  10 = 22/2.2
  11 = (2/2)+(2/.2)
  12 = (2+22)/2
  13 = 2+(22/2)
  14 = 2+(2+(2/.2))
  15 = (2+(2/2))/.2
  16 = 2*(2*(2*2))
  17 = 22-(√.2^-2)
  18 = 22-(2*2)
  19 = (2+(2-.2))/.2
  20 = (2/.2)+(2/.2)
  21 = 22-(2/2)
  22 = 2+(22-2)
  23 = (2/2)+22
  24 = 2*(2+(2/.2))
  25 = 2/(.2*(.2*2))
  26 = 2+(2+22)
  27 = (√.2^-2)+22
  28 = 2+(2+(2*2)!)
  29 = 2+(2+(.2^-2))
  30 = (2+(2*2))/.2
CPU times: user 1.78 s, sys: 16.9 ms, total: 1.8 s
Wall time: 1.8 s


31

# Four 9s

In [24]:
%time show((9, 9, 9, 9))

539,849 entries for (9, 9, 9, 9)

   0 = 99-99
   1 = 99/99
   2 = (99/9)-9
   3 = (9+(9+9))/9
   4 = 9-(9/(.9+.9))
   5 = √9+((9+9)/9)
   6 = ((9+(9+9))/9)!
   7 = 9-((9+9)/9)
   8 = ((9*9)-9)/9
   9 = 9+(9*(9-9))
  10 = 99/9.9
  11 = 9+((9+9)/9)
  12 = (9+99)/9
  13 = √9+(9+(9/9))
  14 = 9+(9/(.9+.9))
  15 = 9+((9+9)/√9)
  16 = 9+((9/.9)-√9)
  17 = 9+(9-(9/9))
  18 = 99-(9*9)
  19 = 9+(9+(9/9))
  20 = 9+(99/9)
  21 = (9+9.9)/.9
  22 = √9+(9+(9/.9))
  23 = √9+((9+9)/.9)
  24 = (99/√9)-9
  25 = √9!+(9+(9/.9))
  26 = (√9*9)-(9/9)
  27 = 9+(9+√(9*9))
  28 = 9+(9+(9/.9))
  29 = 9+((9+9)/.9)
  30 = (9+(9+9))/.9
  31 = (.9+(√9*9))/.9
  32 = (99-√9)/√9
  33 = √9*(99/9)
  34 = (√9+99)/√9
  35 = (√9!+99)/√9
  36 = 9+(9+(9+9))
  37 = (√9*9)+(9/.9)
  38 = √9!*(√9+(√9/.9))
  39 = 9+(√9*(9/.9))
  40 = (9+(√9*9))/.9
  41 = (.9+(√9!*√9!))/.9
  42 = 9+(99/√9)
  43 = √9+(√9!*(√9!/.9))
  44 = (√9!*9)-(9/.9)
  45 = 9*(9/(.9+.9))
  46 = (√9!*√9!)+(9/.9)
  47 = √9*(9+(√9!/.9))
  48 = √9!*(9-(9/9))
  49 = 

62

# Four 5s

In [25]:
%time show((5, 5, 5, 5))

202,937 entries for (5, 5, 5, 5)

   0 = 55-55
   1 = 55/55
   2 = (5/5)+(5/5)
   3 = (5+(5+5))/5
   4 = ((5*5)-5)/5
   5 = 5+(5*(5-5))
   6 = (55/5)-5
   7 = 5+((5+5)/5)
   8 = 5.5+(.5*5)
   9 = 5+(5-(5/5))
  10 = 55/5.5
  11 = 5.5+5.5
  12 = (5+55)/5
  13 = .5+(.5*(5*5))
  14 = 5+((5-.5)/.5)
  15 = (5*5)-(5+5)
  16 = 5+(55/5)
  17 = 5+(5!/(5+5))
  18 = (5-.5)/(.5*.5)
  19 = (5+(5-.5))/.5
  20 = 5+(5+(5+5))
  21 = (5+5.5)/.5
  22 = 55/(.5*5)
  23 = .5+(5*(5-.5))
  24 = (5*5)-(5/5)
  25 = .5*(55-5)
  26 = (5/5)+(5*5)
  27 = (.5*55)-.5
  28 = .5+(.5*55)
  29 = (5!+(5*5))/5
  30 = 55-(5*5)
  31 = 55-(5!/5)
  32 = ((5+5)/5)^5
  33 = .5*(5!*.55)
  34 = 5+(5+(5!/5))
  35 = 5+(5+(5*5))
  36 = (5!+(.5*5!))/5
  37 = (.5^-5)+√(5*5)
  38 = ((5!/5)-5)/.5
CPU times: user 3.33 s, sys: 17.6 ms, total: 3.35 s
Wall time: 3.35 s


39

# Five 5s 

In [26]:
%time show((5, 5, 5, 5, 5), False)

7,624,387 entries for (5, 5, 5, 5, 5)

   0 = 5*(55-55)
   1 = 5^(55-55)
   2 = 55/(.5*55)
   3 = .5*((55/5)-5)
   4 = 5-(55/55)
   5 = 5*(55/55)
   6 = 5+(55/55)
   7 = ((5+55)/5)-5
   8 = .5*(5+(55/5))
   9 = (55-(5+5))/5
  10 = (55/5)-(5/5)
  11 = 5*(55/(5*5))
  12 = (5/5)+(55/5)
  13 = (5+(5+55))/5
  14 = (5*5)-(55/5)
  15 = 5+(55/5.5)
  16 = 5+(5.5+5.5)
  17 = 5+((5+55)/5)
  18 = 5.5+(.5*(5*5))
  19 = (5*5)-(.5+5.5)
  20 = 55/(5*.55)
  21 = 5+(5+(55/5))
  22 = (55+55)/5
  23 = (5+(55/.5))/5
  24 = (5+55)/(.5*5)
  25 = 55-(5+(5*5))
  26 = 5*(5+(5/(5*5)))
  27 = 5+(55/(.5*5))
  28 = .5*(.5+55.5)
  29 = 5+((5*5)-(5/5))
  30 = 5*((55/5)-5)
  31 = .5+(5.5+(5*5))
  32 = (5+(55/5))/.5
  33 = .55*(5+55)
  34 = (5!+(55-5))/5
  35 = 5+(55-(5*5))
  36 = (5*5)+(55/5)
  37 = 5+(((5+5)/5)^5)
  38 = .5+(5*(5+(.5*5)))
  39 = ((5*5)-5.5)/.5
  40 = 55-(5+(5+5))
  41 = (5!*.55)-(5*5)
  42 = (5+5.5)/(.5*.5)
  43 = (.5^-5)+(55/5)
  44 = 55-(55/5)
  45 = (5*5!)-555
  46 = 55+((.5-5)/.5)
  47 = (5*(5+(5

172

# Counting Solutions

Alex Bellos had another challenge:  

> I was half hoping a computer scientist would let me know exactly how many solutions [to the Countdown problem] there are with only the four basic operations. Maybe someone will. 

As it stands, my program can't answer that question, because I only keep one expression for each value. 

Also, I'm not sure what it means to be a distinct solution. For example, are `((10+9)+8)` and `(10+(9+8))` different, or are they same, because they both are equivalent to `(10+9+8)`? Similarly, are `((3-2)-1)` and `(3-(2+1)` different, or the same because they both are equivalent to `(3 + -2 + -1)`? I think the notion of "distinct solution" is just inherently ambiguous. My choice is to count each of these as distinct: every expression has exactly ten numbers, nine operators, and nine pairs of brackets, and if an expression differs in any character, it is different. But I won't argue with anyone who prefers a different definition of "distinct solution."

So how can I count expressions? One approach would be to go back to enumerating every equation (all 4862 &times; 4<sup>9</sup> = 1.2 bilion of them) and checking which ones equal 2016. That would take about 40 hours with my Python program. Another approach is mimic `expressions`, but to make a table of counts, rather than expression strings. I want:

    counts[(10, 9, 8)][27] == 2
    
because there are 2 ways to make 27 with the numbers `(10, 9, 8)`, namely, `((10+9)+8)` and `(10+(9+8))`. And in general, if there are `Lcount` ways to make `L` using `Lnums` and `Rcount` ways to make `R` using `Rnums` then there are `Lcount * Rcount` ways of making `L * R` using `Lnums` followed by `Rnums`. So we have:


In [27]:
@lru_cache()
def counts(numbers):
    "Return a dict of {value: count} for every value that can be made from numbers."
    if len(numbers) == 1: # Only one way to make an expression out of a single number
        return Counter(numbers)
    else: 
        result = Counter()
        for (Lnums, Rnums) in splits(numbers):
            for L in counts(Lnums):
                for R in counts(Rnums):
                    count = counts(Lnums)[L] * counts(Rnums)[R]
                    result[L + R] += count
                    result[L - R] += count
                    result[L * R] += count
                    if R != 0:
                        result[L / R] += count
        return result

In [28]:
counts((2, 2)) # corresponds to {0: '2-2', 1.0: '2/2', 4: '2+2' or '2*2'}

Counter({0: 1, 1.0: 1, 4: 2})

In [29]:
counts((10, 9, 8))[27]

2

Looks good to me. Now let's see if we can answer the question.

In [30]:
counts(c10)[2016]

30066

This says there are 30,066 distinct expressions for 2016. 

**But we're forgetting about round-off error.**

Let's find all the values that are very near to `2016`:

In [31]:
{y: counts(c10)[y] 
 for y in counts(c10)
 if abs(y - 2016) < 1e-10}

{2015.999999999997: 15,
 2015.999999999999: 10,
 2015.9999999999993: 14,
 2015.9999999999995: 1930,
 2015.9999999999998: 5868,
 2016.0: 30066,
 2016.0000000000002: 5792,
 2016.0000000000005: 510,
 2016.0000000000018: 264,
 2016.000000000002: 18,
 2016.0000000000023: 12}

I suspect that all of these actually should be exactly 2016. 
To be absolutely sure, I could re-do the calculations using exact rational arithmetic, as provided by the `fractions.Fraction` data type. From experience I know that would be an order of magnitude slower, so instead I'll just add up the counts:

In [32]:
sum(_.values())

44499

I have more confidence in this answer, 44,499, than in 30,066, but I wouldn't accept it as definitive until it was independently verified and passed an extensive test suite. And of course, if you have a different definition of "distinct solution," you will get a different answer.


# Countdown to 2018

One  more thing: On January 1 2018, [Michael Littman](http://cs.brown.edu/~mlittman/) posted this:

> 2+0+1×8, 2+0-1+8, (2+0-1)×8, |2-0-1-8|, -2-0+1×8, -(2+0+1-8), sqrt(|2+0-18|), 2+0+1^8, 20-18, 2^(0×18), 2×0×1×8... Happy New Year!

Can we replicate that countdown?

In [33]:
def littman_countdown(year):
    "Return a Littman countdown for the year, given as a tuple of numbers."
    return ', '.join(unbracket(expressions(year)[i])
                     for i in reversed(range(11)))

print(littman_countdown((2, 0, 1, 8)), '... Happy New Year!')

2.0+(1*8), (2*0)+(1+8), (2*0)+(1*8), ((2*0)-1)+8, (2*(0-1))+8, -2+((0-1)+8), 20*(1-.8), 2.0+(1^8), 20-18, 2^(0*18), 2*(0*18) ... Happy New Year!


Similar results, with some alternatives for some numbers. Let's get ready for next year:

In [34]:
print(littman_countdown((2, 0, 1, 9)), '... Happy New Year!')

20-(1+9), (2*0)+(1*9), ((2*0)-1)+9, (2*(0-1))+9, -2+((0-1)+9), 20/(1+√9), (2*0)+(1+√9), 2+(0.1+.9), 2+(0*19), 20-19, 2*(0*19) ... Happy New Year!


# What's Next?

One exercise would be adding even more operators, such as:

- **Floor and Ceiling**: `⌊5.5⌋ == 5, ⌈5.5⌉ == 6`
- **Nth root**: `3√8 == 2`
- **Percent**: `5% == 5/100`
- **Repeating decimal**: `.4_ == .44444444... = 4/9`
- **Transcendental functions**: `log, sin, cos, tan, arcsin, ...` 

What would you like to do?